In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [35]:
#READ

#read data
df = pd.read_csv("data.csv")


#select data
selected = ["Weapon", "Range", "DropOff", 
            "Zero Damage Range", "RPM","Capacity","ReloadSpeedFromEmpty(ms)", 
            "HeadshotMultiplier", "WeaponBonus", "MaxBonus%", "regularDMG", "minimumDMG"]
df = df[selected]
#CLEAN

#replacement
df['ReloadSpeedFromEmpty(ms)'] = df['ReloadSpeedFromEmpty(ms)'].str.replace("-", "0").astype(str)

#Biased Weapon bonuses
"""
Health: Health Damage
CHD: Critical Hit Damage
HSD: Headshot Damage
Armor: Armor Damage
OOC: DMG To Target Out Of Cover
CHC: Critical Hit Chance

Personal preference towards Armor, HSD, CHD

The weights are from 0-600. Increment by 100 so it doesn't get lost. I like Armor more than HSD, so 100 difference
"""
df["WeaponBonus"] = df["WeaponBonus"].str.replace("Health", "200").astype(str)
df["WeaponBonus"] = df["WeaponBonus"].str.replace("CHD", "400").astype(str) 
df["WeaponBonus"] = df["WeaponBonus"].str.replace("HSD", "500").astype(str) 
df["WeaponBonus"] = df["WeaponBonus"].str.replace("Armor", "600").astype(str) 
df["WeaponBonus"] = df["WeaponBonus"].str.replace("OOC", "100").astype(str)
df["WeaponBonus"] = df["WeaponBonus"].str.replace("CHC", "300").astype(str) 
df["WeaponBonus"] = df["WeaponBonus"].str.replace("-", "0").astype(str)

#bonus% fix
df['MaxBonus%'] = df['MaxBonus%'].str.replace('%', '').astype(str)
df['MaxBonus%'] = df['MaxBonus%'].str.replace('-', '0').astype(str)
#headshot multiplier fix
df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1 \+ Bonus', '1',regex=True).astype(str)
df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1\+bonus', '1',regex=True).astype(str) #this also happens

#type setting. (Just in case it needs to be done)
df = df.astype({"Weapon": str, "Range": int, "DropOff": int, 
            "Zero Damage Range": int, "RPM": int,"Capacity": int,"ReloadSpeedFromEmpty(ms)": int, 
            "HeadshotMultiplier": float, "WeaponBonus": int, "MaxBonus%": int, "regularDMG": int, "minimumDMG": int})


In [30]:
#dictionary of weights to be multiplied.
#I am biasing high DMG, high RPM. 

#Biased against long reloads

#bonus is liked, but not as much as others
weights = {
    "highDMG": 0.4,
    "RPM": 0.3,
    "reload": -0.1,
    "bonus": 0.2,
    "wpnBonus": 0.2
}

### Normalize, Multiply weights, score

In [31]:
#NORMALIZE
df["normDMG"] = df["regularDMG"] / df["regularDMG"].max()
df["normRPM"] = df["RPM"] / df["RPM"].max()
df["normReload"] = df["ReloadSpeedFromEmpty(ms)"] / df["ReloadSpeedFromEmpty(ms)"].max()
df["normMinDMG"] = df["minimumDMG"] / df["minimumDMG"].max()
df["normBonus"] = df["MaxBonus%"] / df["MaxBonus%"].max()
df["normWPN"] = df["WeaponBonus"] / df["WeaponBonus"].max()



In [32]:
#SCORE WEIGHTED

#weights of the damage
weightDMG = (weights["highDMG"] * df["normDMG"])

#weight of the RPM and reload
weightRPM = (weights["RPM"] * df["normRPM"]) + (weights["reload"] * df["normReload"])

#weight of bonus
weightBonus = (weights["bonus"] * df["normBonus"]) + (weights["wpnBonus"] * df["normWPN"])

#total
df["scored_weight"] = weightDMG + weightRPM + weightBonus

In [33]:
first = df.sort_values(by="scored_weight", ascending=False).iloc[0]
second = df.sort_values(by="scored_weight", ascending=False).iloc[1]
third = df.sort_values(by="scored_weight", ascending=False).iloc[2]

print(first)
print(second)
print(third)

Weapon                       Nemesis
Range                             60
DropOff                           80
Zero Damage Range                100
RPM                               50
Capacity                           5
ReloadSpeedFromEmpty(ms)        3400
HeadshotMultiplier               1.0
WeaponBonus                      500
MaxBonus%                        111
regularDMG                    940240
minimumDMG                    940240
normDMG                          1.0
normRPM                     0.041667
normReload                       0.4
normMinDMG                       1.0
normBonus                   0.810219
normWPN_Bonus               0.833333
normWPN                     0.833333
scored_weight                0.70121
Name: 85, dtype: object
Weapon                      The White Death - Classic M44 Carbine
Range                                                          60
DropOff                                                       100
Zero Damage Range                     